# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [2]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q datasets

## 데이터셋: 직접 annotation한 review데이터

In [2]:
from datasets import Dataset

# CSV 파일 로드
dataset = Dataset.from_csv('./data/review summary prototype dataset 0707.csv')

# 데이터 확인
print(dataset)

Found cached dataset csv (/opt/ml/.cache/huggingface/datasets/csv/default-62da9c4cc6019130/0.0.0)


Dataset({
    features: ['id', 'search_name', 'instruction', 'input', 'output'],
    num_rows: 107
})


In [3]:
dataset

Dataset({
    features: ['id', 'search_name', 'instruction', 'input', 'output'],
    num_rows: 107
})

In [4]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = dataset.map(
    lambda x: {'text': f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: {x['instruction']} ### 입력: {x['input']} ### 응답: {x['output']}" }
)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/csv/default-62da9c4cc6019130/0.0.0/cache-51551d8543171dc8.arrow


## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-5.8b"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAAAgQDaf2gtrNKbTR7WVbkc2Buupn223FNv24qXf4MRD2I0WIYCl3rV0caz0aVR7qygb0iyGVnxVh8g2c7gDaixAN2o0eW91u7FPzQHarZVCULL2g19jGlbhLWT8sAaxjbOWgtsptV571+mqllm/otr2Q2Ao2a+eKNvL0osGbWpu5tfGw==')}
  warn(msg)
/opt/conda/lib/python3.8/site-packages/bitsandbytes/cuda_setu

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [6]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

In [7]:
data

Dataset({
    features: ['id', 'search_name', 'instruction', 'input', 'output', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 107
})

In [8]:
data['text'][0]

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: 다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다 ### 응답: <양> 1패키지에 2 3인분 용량 <양> 1팩당 1165 칼로리 <재료> 밀떡 <조리> 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 <보관> 냉동보관'

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


In [12]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Jul 10 12:45:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:05.0 Off |                  Off |
| N/

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [13]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # max_steps=50, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        num_train_epochs=1,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.616400
20,2.313600
30,2.003900
40,1.900100
50,1.938300


TrainOutput(global_step=54, training_loss=2.1261529657575817, metrics={'train_runtime': 44.3586, 'train_samples_per_second': 2.412, 'train_steps_per_second': 1.217, 'total_flos': 686368025640960.0, 'train_loss': 2.1261529657575817, 'epoch': 1.0})

In [14]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!a

In [15]:
model.generate(**tokenizer("### 명령어: 다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 247, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.t

tensor([[    6,     6,     6,  5026,   348,    29,  1462,   285, 19903,   274,
           741, 19310,   301,   422, 15503,    33,  5272, 16041,  4514,   339,
         12670,   284,  1934,  6501,     6,     6,  9272,    29,  5186,   285,
          1182,  1305, 19903, 15672, 12232,  5873,    74,   401,   533, 12211,
          7566,   807,  1779,  1929,   440,  1739,    23,   561,  1780,   393,
          2764, 13013,  1780,   440,  1739,    22,   561,   678,   393, 17306,
         13473, 10967, 29468,  7566,   339,  5559,   284,   379,  7566,   605,
           450,  1173,  6639,   288, 12232, 12189,  7759,  1396,   693,  6919,
           476, 11747,  5019,  1253, 19903,  3100,   296,   749,   926,   270,
           750,  3013,   376, 20764,   274,   401,   524, 25034,  9588,  1441,
          5873,    74,    20,  5092,   519,  1132,  4901,  3476,  9086,   828,
           272, 19903,  1678,  6310,   812,  5283,  3305,   284,   453,  1330,
           450,  3013,  9664,   240,  9664,   240,  

In [16]:
def gen(x,y):
    gened = model.generate(
        **tokenizer(
            f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: {x} ### 입력: {y} ### 응답:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [17]:
gen('다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요', '추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: 다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다 ### 응답: <후기> 냉동보관 제품의 조리 시 유의점 조리 시 팁을 상세하게 적어주셔서 감사합니다 >> 조리 시 유의점 냉장보관을 해야 하는 상품이 아니기 때문에 배송 시에 배송일로부터 4일이 지나면 먹기에 조금 시큼할 수 있는 점 참조 꼭 냉동보관 하시어 빠르면 다음날 조리해서 먹으면 더욱 맛있게 먹을 수 있다는 거 유의해 주세요 밀떡의 특성이 냉동보관해도 쫄깃하다는 장점이 있어요 냉동한 떡을 찬물에 담궈 해동을 한 후 손으로 떼어 넣는대요 저는 그런 과정을 생략했지만 그래도 잘 만들면 맛있어요 저는 오뎅은 없지만 이것저것 넣어서 먹으면 더욱 맛있는 떡볶이가 되었을거에요 >> 조리 시 팁 냉동된 떡을 찬물에 담그거나 찬물을 한컵 넣어 해동을 하면 떡이 쫄깃쫄기 할 수 있어요 먹기 알맞게 떼어서 넣어도 되시구요 >> 배송 시 유의점 냉동보관 제품으로 보관법에 따라 냉동보관해주셔야 하는 제품이에요 가급적

`아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: 다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다`

`### 응답: <후기> 냉동보관 제품의 조리 시 유의점 조리 시 팁을 상세하게 적어주셔서 감사합니다 >> 조리 시 유의점 냉장보관을 해야 하는 상품이 아니기 때문에 배송 시에 배송일로부터 4일이 지나면 먹기에 조금 시큼할 수 있는 점 참조 꼭 냉동보관 하시어 빠르면 다음날 조리해서 먹으면 더욱 맛있게 먹을 수 있다는 거 유의해 주세요 밀떡의 특성이 냉동보관해도 쫄깃하다는 장점이 있어요 냉동한 떡을 찬물에 담궈 해동을 한 후 손으로 떼어 넣는대요 저는 그런 과정을 생략했지만 그래도 잘 만들면 맛있어요 저는 오뎅은 없지만 이것저것 넣어서 먹으면 더욱 맛있는 떡볶이가 되었을거에요 >> 조리 시 팁 냉동된 떡을 찬물에 담그거나 찬물을 한컵 넣어 해동을 하면 떡이 쫄깃쫄기 할 수 있어요 먹기 알맞게 떼어서 넣어도 되시구요 >> 배송 시 유의점 냉동보관 제품으로 보관법에 따라 냉동보관해주셔야 하는 제품이에요 가급적 빨리 드시고 조리법대로 조리하여 드시는 것을 추천합니다 신선하게 빨리 먹는 것이 가장`

In [23]:
import time
import datetime # datetime 라이브러리 import
 
start = time.time() # 시작
gen('다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요', '추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다')
sec = time.time()-start # 종료 - 시작 (걸린 시간)
 
times = str(datetime.timedelta(seconds=sec)) # 걸린시간 보기좋게 바꾸기
short = times.split(".")[0] # 초 단위 까지만
print(f"{short} sec")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: 다음의 떡볶이에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 추억의 국민학교 떡볶이 오리지널 냉동 600g 2개 후기 배송날짜 23년 04월 24일 유통기한 24년 03월 20일주문하여 로켓프레시 배송으로 신선하게 배송되었어요 간편한 냉동보관 조리 제품 올인원 다 때려넣으면 떡볶이 완성은근 양이 많네요 1패키지에 2 3인분 용량입니다 600g1팩당 1165칼로리저는 떡볶이 밀떡파라서 만족하며 먹었네요ㅎㅎ조리방법대로 10분 정도 떡을 찬물에 담군 후에 손으로 떼어주었어요 보통 같으면 당면이나 파 양파등 다양한 부재료를 손질하여 해먹었을텐데배가 고파서 아무것도 준비 안하고 먹었네요ㅎㅎ냉동보관 제품으로 아무때나 꺼내 먹을 수 있는게 가장 큰 장점입니다배송 감사합니다 많이 파세요저도 상품을 구매할때 리뷰로 도움을 많이 받은만큼솔직하게 적어보았습니다도움이 되었다면 도움이 돼요 꾹 눌러주시면 감사하겠습니다 ### 응답: <조리> 간편하 냉동보관 조리 제품 <양> 1패키지에 2 3인분 용량 <재료> 냉동 보관 제품 <맛> 냉동 보관 제품인데도 조리하고 나니 쫄깃쫄깃 합니다 <조리> 냉동 보관 제품이라서 조리해 먹을 수 있게 나오는것도 간편해서 밀떡이라서 매우 만족하며 먹었네요ㅎㅎ <재료> 냉동 보관 제품이라 보관하기 매우 편리하고 유통기한도 넉넉 합니다 맛은 밀떡이라서 쫄깃쫄깃 하네요 보통 같으면 당면이나 파 양파등을 준비해서 먹거나 어묵을 넣어서 먹었을텐데 배가 고파서 아무것도 준비안하고 먹었네요ㅎㅎ <조리> 간편하게 조리할 수 있는 제품이라서 매우 만족하며 먹었네요ㅎㅎ 하지만 조리할때는 조리 방법대로 조리해야 맛이 있으므로 조리방법대로 꼭 조리하세요 10분 정도 떡을 찬물에 담근 뒤 손으로 떼어서 손으로 직접 떼어내도 되지만 저는 떡이 흐물거리는게 싫어서 조리용기에 물을 넣고 그대로 전자렌지에 넣어서 조

In [18]:
gen('다음의 치킨에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요', '우선 에어프라이어 특화상품이라 편해요어떻게 조리할 지 고민할 필요 없어요에어프라이기를 180190도로 예열한 다음 같은 온도에서 10분 조리하면 끝이에요처음에 먹을 때는 10분만 돌렸을 때 고기가 덜 익어서 추가로 몇 분 더 돌려줬어요다음에 먹을 때는 예열 시간을 더 줬더니 치킨 넣고 10분만 돌려도 충분했어요순살치킨에 뻥튀기 같은 라이스볼이 붙어 있어서 식감이 더욱 좋아요전분도 로스팅 공법으로 만들어 바삭함을 높였다고 하구요녹차유로 숙성까지 했다고 하네요따로 간을 하지 않아도 꽤 짭짤한 편이에요물론 취향에 따라서는 소스를 찾을 수도 있지만요닭고기는 닭다리살이고 태국 브라질 덴마크 산으로 표기되었어요참고로 에어프라이어 외에 프라이팬 튀김냄비 조리법도 나와있으니어떤게 더 맛있나 비교해봐도 좋을 것 같아요다른 두 가지 방법은 상당한 양의 식용유를 필요로 하지만요')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: 다음의 치킨에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 우선 에어프라이어 특화상품이라 편해요어떻게 조리할 지 고민할 필요 없어요에어프라이기를 180190도로 예열한 다음 같은 온도에서 10분 조리하면 끝이에요처음에 먹을 때는 10분만 돌렸을 때 고기가 덜 익어서 추가로 몇 분 더 돌려줬어요다음에 먹을 때는 예열 시간을 더 줬더니 치킨 넣고 10분만 돌려도 충분했어요순살치킨에 뻥튀기 같은 라이스볼이 붙어 있어서 식감이 더욱 좋아요전분도 로스팅 공법으로 만들어 바삭함을 높였다고 하구요녹차유로 숙성까지 했다고 하네요따로 간을 하지 않아도 꽤 짭짤한 편이에요물론 취향에 따라서는 소스를 찾을 수도 있지만요닭고기는 닭다리살이고 태국 브라질 덴마크 산으로 표기되었어요참고로 에어프라이어 외에 프라이팬 튀김냄비 조리법도 나와있으니어떤게 더 맛있나 비교해봐도 좋을 것 같아요다른 두 가지 방법은 상당한 양의 식용유를 필요로 하지만요 ### 응답: <조리> 편해요 <재료> 어떤 재료로 구성되어 있나 고민할 필요 없어요 <조리> 예열시간을 더 줘서 구우면 더욱 바삭하다고 하네요 <맛> 짭짤함 맛 > 간을 하지 않아도 꽤 짭짤한 편이에요 <식감> 로스팅 공법으로 바삭함 강조하였다고 해요 <특징> 녹차유로 숙성까지 해서 더욱 풍미가 깊다고 하네요 <영양> 닭고기는 닭다리살로 기름기가 있는 편이라고 하네요 <조리> 간편해요 <입맛> 기호에 따라서는 소스를 찾을 수 있지만요 <평가> 취향에 따라 선호가 달라져요 <재료> 닭안심살 <조리> 에어프라이어 <식감> 예열시간 더 줘서 구우면 바삭하다고 하네요 <식맛> 짭짤한 것이 취향에 따라 더 선호하게 되는 것 같아요 <식성> 간에 따라서 선호도 변하기도 하고요 <조리> 간편해요 <재료> 어떤 재료로 구성되어 있나 고민할 필요 없어요 <조리> 예열시간을 더 줘서 구우면 더욱 바

`아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### 명령어: 다음의 치킨에 대한 리뷰를 <특성> 세부설명 형식으로 요약하세요 ### 입력: 우선 에어프라이어 특화상품이라 편해요어떻게 조리할 지 고민할 필요 없어요에어프라이기를 180190도로 예열한 다음 같은 온도에서 10분 조리하면 끝이에요처음에 먹을 때는 10분만 돌렸을 때 고기가 덜 익어서 추가로 몇 분 더 돌려줬어요다음에 먹을 때는 예열 시간을 더 줬더니 치킨 넣고 10분만 돌려도 충분했어요순살치킨에 뻥튀기 같은 라이스볼이 붙어 있어서 식감이 더욱 좋아요전분도 로스팅 공법으로 만들어 바삭함을 높였다고 하구요녹차유로 숙성까지 했다고 하네요따로 간을 하지 않아도 꽤 짭짤한 편이에요물론 취향에 따라서는 소스를 찾을 수도 있지만요닭고기는 닭다리살이고 태국 브라질 덴마크 산으로 표기되었어요참고로 에어프라이어 외에 프라이팬 튀김냄비 조리법도 나와있으니어떤게 더 맛있나 비교해봐도 좋을 것 같아요다른 두 가지 방법은 상당한 양의 식용유를 필요로 하지만요`

`### 응답: <조리> 편해요 <재료> 어떤 재료로 구성되어 있나 고민할 필요 없어요 <조리> 예열시간을 더 줘서 구우면 더욱 바삭하다고 하네요 <맛> 짭짤함 맛 > 간을 하지 않아도 꽤 짭짤한 편이에요 <식감> 로스팅 공법으로 바삭함 강조하였다고 해요 <특징> 녹차유로 숙성까지 해서 더욱 풍미가 깊다고 하네요 <영양> 닭고기는 닭다리살로 기름기가 있는 편이라고 하네요 <조리> 간편해요 <입맛> 기호에 따라서는 소스를 찾을 수 있지만요 <평가> 취향에 따라 선호가 달라져요 <재료> 닭안심살 <조리> 에어프라이어 <식감> 예열시간 더 줘서 구우면 바삭하다고 하네요 <식맛> 짭짤한 것이 취향에 따라 더 선호하게 되는 것 같아요 <식성> 간에 따라서 선호도 변하기도 하고요 <조리> 간편해요 <재료> 어떤 재료로 구성되어 있나 고민할 필요 없어요 <조리> 예열시간을 더 줘서 구우면 더욱 바삭하다고 하네요 <특징> 녹차유로 숙성까지 해서 더욱 풍미가 깊다고 하네요`

## 모델 저장 & 업로드

In [22]:
# huggingface-cli login

In [20]:
model.push_to_hub('boostcamp-5th-nlp07/qlora-koalpaca-polyglot-5.8b-fast')

adapter_model.bin:   0%|          | 0.00/14.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/boostcamp-5th-nlp07/qlora-koalpaca-polyglot-5.8b-fast/commit/ae70717d9ea726d858c6dddab3c432d7b0cc9f9f', commit_message='Upload model', commit_description='', oid='ae70717d9ea726d858c6dddab3c432d7b0cc9f9f', pr_url=None, pr_revision=None, pr_num=None)

# Hugginface Hub에 올린 LoRA Weight 저장한것 불러오는 튜토리얼 코드

- HF에 업로드 후, 실제 다운받아 사용시 쓰는 코드 예제입니다.

--> https://github.com/Beomi/KoAlpaca/blob/main/2023_06_08_PEFT%EB%A1%9CLoRA%EB%A1%9C%EB%93%9C.ipynb